# Занятие 7: Внешнее соединение JOIN

_TODO дополнить про интерфейс и запуск БД_

_а пока воспользуйтесь следующей ссылкой_<br>
http://84.201.167.29/phpmyadmin<br>
пользователь: student<br>
пароль: n123456789

В последнем наборе упражнений мы сосредоточились на внутренних соединениях.  В большинстве случаев внутренние соединения дадут вам результаты, которые вы ищете.  Однако иногда вам может потребоваться включить все данные из таблицы в свои вычисления или выходные данные, даже если не все эти данные совпадают с данными из других таблиц, к которым вы присоединяетесь.  
    
Например, если у вас есть таблица с демографической информацией о клиентах и таблица с информацией о том, каким клиентам был отправлен бесплатный образец, возможно, вам захочется проанализировать характеристики клиентов, которые получили и не получили образец.  Лучшим способом сделать это в такой программе, как Tableau, было бы поместить всю информацию о вашем клиенте в одну таблицу с дополнительным столбцом, указывающим, получил ли клиент бесплатный образец или нет.  В качестве альтернативы, вы можете сгенерировать список клиентов, которые * не * получили бесплатный образец, чтобы вы могли организовать получение клиентами бесплатного образца в будущем.

В подобных ситуациях вы будете использовать внешние соединения для соединения таблиц.  Внешние соединения включают левые соединения, правые соединения или полные внешние соединения (напомним, что полные внешние соединения не поддерживаются в MySQL).  Смотрите видео "Что такое объединения?" и "Соединения со связями "Многие ко многим" и дубликатами" для получения дополнительной информации о соединениях. 

Вот картинка, которая напомнит вам об общих концепциях, лежащих в основе внешних соединений:

<img src="https://duke.box.com/shared/static/4s89rmm8a75tep4puyqt1tdlhic0stzx.jpg" width=400 alt="SELECT FROM WHERE" />



## Внешние соединения RIGHT/LEFT JOIN 

## Left and Right Joins

Левое и правое соединения используют синтаксис, отличный от того, который мы использовали в уроке о внутренних соединениях.  Метод, который я показал вам для выполнения внутренних соединений, сообщает базе данных, как связать таблицы в предложении WHERE следующим образом:

```mySQL
WHERE d.dog_guid=r.dog_guid
```

В случае соединений через указание двух таблиц мы ищем полное соответствие значений. Однако мы можем переписать внутренние соединения в том же синтаксисе, который используется внешними соединениями. Чтобы использовать этот более традиционный синтаксис, вы должны указать базе данных, как подключать таблицы, используя предложение ON, которое следует сразу за предложением FROM.  Обязательно укажите слово "JOIN" явно.  Эта традиционная версия синтаксиса освобождает предложение WHERE для других элементов, которые вы, возможно, захотите включить в свой запрос.  Вот как выглядел бы один из наших запросов из занятия 6 с использованием традиционного синтаксиса:

```mySQL
SELECT d.dog_guid AS DogID, d.user_guid AS UserID, AVG(r.rating) AS AvgRating, COUNT(r.rating) AS NumRatings, d.breed, d.breed_group, d.breed_type
FROM dogs d JOIN reviews r
  ON d.dog_guid=r.dog_guid AND d.user_guid=r.user_guid
GROUP BY d.user_guid
HAVING NumRatings > 9
ORDER BY AvgRating DESC
LIMIT 200
```

Вы также могли бы написать "INNER JOIN" вместо "JOIN", но по умолчанию в MySQL JOIN будет означать внутреннее соединение, поэтому включение слова "INNER" необязательно.

Если вам нужно предложение WHERE в приведенном выше запросе, оно должно быть после предложения ON и перед предложением GROUP BY.

Вот пример другого запроса, который мы использовали на прошлом уроке и в котором использовался синтаксис с простым указанием двух таблиц:

```mySQL
SELECT d.user_guid AS UserID, d.dog_guid AS DogID, 
       d.breed, d.breed_type, d.breed_group
FROM dogs d, complete_tests c
WHERE d.dog_guid=c.dog_guid AND test_name='Yawn Warm-up';
```

**Question 1: How would you re-write this query using the traditional join syntax?**

### Вопрос 1.

**Как записать запрос выше, используя JOIN?**

Теперь, когда мы ознакомились с синтаксисом соединения, мы можем начать практиковать внешние соединения. Наш набор данных распознавания сделает внешние объединения более сложными, чем обычно, из-за отсутствия объявленных первичных ключей в исходной базе данных, связей "многие ко многим" и наличия повторяющихся строк и нулевых значений в столбцах, которые мы будем использовать для объединения таблиц.  Однако освоение внешних объединений в этом сложном контексте гарантирует, что вы поймете фундаментальные концепции, лежащие в основе объединений, что станет огромным преимуществом, когда вы начнете писать запросы в базы данных других компаний.

Первый запрос, который я описал выше, изначально был написан для ответа на вопрос о том, владельцы каких пород оставляют больше всего отзывов. Когда мы разрабатывали этот запрос на прошлом уроке, мы хотели сосредоточиться на владельцах собак, которые оставили отзыв не менее 10 раз для одной или нескольких своих собак.  Мы также хотели узнать породу, breed_type и breed_group каждой из собак этих владельцев.  

Изучая запрос, мы узнали, что:

+ Все user_guids в таблице отзывов находятся в таблице dogs
+ Только 389 из более чем 5000 dog_guids в таблице отзывов находятся в таблице dogs

Внутреннее соединение, которое мы выполнили, привело к получению 389 строк выходных данных, поскольку оно включало только данные из строк, которые имеют эквивалентные значения в обеих объединяемых таблицах.  Но что, если бы мы хотели получить полный список собак в таблице отзывов и указание на то, были ли они в таблице dogs или нет, а не только список информации об отзывах от собак в таблице dogs?  Чтобы получить этот список, мы могли бы выполнить внешнее соединение.  
     
<img src="https://i.stack.imgur.com/1UKp7.png" width=600 />

Давайте начнем с использования левого внешнего соединения, чтобы получить нужный нам список. Когда мы используем традиционный синтаксис join для написания внутренних соединений, порядок ввода таблиц в запросе не имеет значения.  Однако во внешних соединениях порядок имеет большое значение.  Левое внешнее соединение будет включать в себя все строки таблицы слева от предложения LEFT JOIN. Правое внешнее соединение будет включать в себя все строки таблицы справа от предложения RIGHT JOIN.  Таким образом, чтобы получить полный список собак, которые прошли по крайней мере 10 тестов в таблице отзывов, и включить как можно больше информации о породе, мы могли бы запросить:


```mysql
SELECT r.dog_guid AS rDogID, d.dog_guid AS dDogID, r.user_guid AS rUserID, d.user_guid AS dUserID, AVG(r.rating) AS AvgRating, COUNT(r.rating) AS NumRatings, d.breed, d.breed_group, d.breed_type
FROM reviews r LEFT JOIN dogs d
  ON r.dog_guid=d.dog_guid AND r.user_guid=d.user_guid
WHERE r.dog_guid IS NOT NULL
GROUP BY r.dog_guid
HAVING NumRatings >= 10
ORDER BY AvgRating DESC;
```



### Вопрос 2.

**Как записать запрос выше, используя RIGHT JOIN?**

Обратите внимание, что в выходных данных как левой, так и правой версии внешнего соединения все строки, которые имели dog_guid в таблице reviews, но не имели соответствующего dog_guid в таблице dogs, имеют слово "Нет", введенное в выходных столбцах, связанных с таблицей dogs.  "NULL", в данном случае, что значение отсутствует.  Это становится ясно, когда вы запрашиваете список только тех dog_guides, которых не было в таблице dogs:

```mysql
SELECT r.dog_guid AS rDogID, d.dog_guid AS dDogID, r.user_guid AS rUserID, d.user_guid AS dUserID, AVG(r.rating) AS AvgRating, COUNT(r.rating) AS NumRatings, d.breed, d.breed_group, d.breed_type
FROM reviews r LEFT JOIN dogs d
  ON r.dog_guid=d.dog_guid AND r.user_guid=d.user_guid
WHERE d.dog_guid IS NULL
GROUP BY r.dog_guid
HAVING NumRatings >= 10
ORDER BY AvgRating DESC;
```


### Вопрос 3.

**Как бы вы использовали левое соединение LEFT JOIN, чтобы получить список всех уникальных идентификаторов питомцев `dog_guid` в таблице `dogs` и подсчитать, сколько тестов `test_name` выполнила каждая из них? Включите в свои выходные данные `dog_guid` и `user_guid` из таблиц `dogs` и `complete_tests`.**

Иногда вы можете быть настолько сосредоточены на написании своего заявления о соединении, что не обращаете пристального внимания на поля и таблицы, которые вы добавляете в другие свои предложения, особенно когда вы объединяете много таблиц.  Часто ваш запрос все равно будет выполнен успешно, даже если вы не ввели критерии или предложение группировки, которые вы намеревались ввести.  Следующий вопрос проиллюстрирует, насколько легко это происходит.

### Вопрос 4.

**Повторите запрос, который вы выполнили в вопросе 3, но используйте `dog_guid` из таблицы `completed_tests` для группировки ваших результатов вместо таблицы dogs.**

На этот раз ваш запрос был выполнен успешно, но вы получили гораздо меньше идентификаторов DOGID, потому что предложение GROUP BY сгруппировало ваши результаты в соответствии с идентификаторами dog_guid в таблице completed_tests, а не с таблицей dog_guid.  В результате, несмотря на то, что вы правильно реализовали свое соединение, все dog_guid, которые были в таблице dogs, но не в таблице completed_tests, были сведены в одну строку вашего вывода, где completed_tests.dogs_guid = NULL.  Это хорошая возможность напомнить себе о различиях между SELECT/GROUP BY и COUNT DISTINCT. 

```MySQL
SELECT COUNT(DISTINCT dog_guid)
FROM complete_tests;
```

Результатом вашего предложения COUNT DISTINCT должно быть 17 986, что на одну строку меньше, чем количество строк, которые вы извлекли из своего запроса в вопросе 4.  Это потому, что COUNT DISTINCT НЕ подсчитывает значения NULL, в то время как предложения SELECT/GROUP BY объединяют значения NULL в одну группу.  <отметка> Если вы хотите вывести количество отдельных записей из результатов запроса, используя предложения joins и GROUP BY, не забудьте включить предложение "IS NOT NULL", чтобы убедиться, что вы не подсчитываете значения NULL.</mark>
   
Эти упражнения являются хорошей иллюстрацией того, почему очень полезно сохранять ваши запросы, когда вы проводите анализ.  Сохранение ваших запросов позволит вам и членам вашей команды перепроверить вашу работу позже.  Как вы можете видеть, концепции, лежащие в основе SQL, сами по себе не слишком сложны, но легко допустить ошибки, особенно когда ваши запросы становятся длиннее и сложнее.  

Еще одна ситуация, когда объединения могут вызвать некоторую путаницу, - это когда у вас есть повторяющиеся строки в таблице, к которой вы присоединяетесь.  Если вы проигнорируете то, что мы обсуждали о теории множеств и о том, как базы данных вычисляют свои объединения, поведение баз данных, проявляющееся при наличии повторяющихся строк в объединенной таблице, покажется совершенно непонятным.  Однако, обладая этим знанием, поведение будет иметь полный смысл.  Давайте пройдемся по тому, что происходит. 


### Вопрос 5.

**Как извлечь всю информацию о породе каждой собаки, которой владеет пользователь `user_guid` в таблице `users`.  Если у `user_guid` в таблице `users` нет собаки, нам также нужна эта информация. Включите в свой вывод `dog_guid` из таблицы `dogs` и `user_guid` как из таблиц `users`, так и из таблиц `dogs`.**

В таблице dogs всего 350 различных dog_guides.  Почему база данных выдает почти миллион строк?  Этого не может быть.  Давайте разберемся, что происходит.

### Вопрос 6.

**Адаптируйте запрос, который вы написали выше, таким образом, чтобы он подсчитывал количество строк, которые будет выводить соединение для каждого `user_id`.  Отсортируйте результаты по этому показателю в порядке убывания.  Помните, что если вы включите поля `dog_guid` или `breed` в этот запрос, они будут случайным образом заполнены только одним из значений, связанных с `user_guid`.**

Этот запрос сообщил нам, что пользователь 'ce7b75bc-7144-11e5-ba71-058fbc01cf0b' будет связан с 913 138 строками в выходных данных внешнего соединения, которое мы разработали!  Еще раз, почему?  Мы собираемся поработать со вторым user_guid в выходных данных, которые вы только что сгенерировали, 'ce225842-7144-11e5-ba71-058fbc01cf0b', потому что он будет связан с 442 выходными строками, а с 442 строками работать намного проще, чем с 913 138.

Сколько строк в таблице *users* связано с user_guid 'ce225842-7144-11e5-ba71-058fbc01cf0b'?

```MySQL
SELECT COUNT(*)
FROM users
WHERE user_guid='ce225842-7144-11e5-ba71-058fbc01cf0b';
```

В таблице users есть 17 записей, связанных с этим user_guid. Если вы изучите все столбцы в записях, вы увидите, что строки являются точными дубликатами друг друга. Это прискорбно, но также может произойти и с реальными наборами данных, особенно от новых компаний или правительственных учреждений.

Проверим все строки в таблице *dogs*, которые связаны с user_guid 'ce225842-7144-11e5-ba71-058fbc01cf0b'?

```MySQL
SELECT COUNT(*)
FROM dogs
WHERE user_guid='ce225842-7144-11e5-ba71-058fbc01cf0b';
```

Вы должны увидеть, что в таблице dogs есть 26 строк, связанных с этим идентификатором пользователя.  Когда вы изучаете таблицу собак, вы видите, что есть много записей, в которых в графе порода указано "Ши-тцу", а в графе вес - "190".  Это было внутреннее соглашение Dognition для указания тестовых учетных записей.  Таким образом, эти dog_guides и user_guides не представляют реальных данных.  Тем не менее, они являются отличным примером того, что происходит, когда вы объединяете поля, содержащие повторяющиеся записи.

Вспомним общую стратегию, используемую реляционными базами данных для объединения таблиц:

<img src="https://duke.box.com/shared/static/km5c7scvo7u6aexzsy1i73wm28aizme1.jpg" width=400 alt="SELECT FROM WHERE" />


Когда базы данных объединяют таблицы, они выводят результат каждой пары записей, удовлетворяющих определенным критериям, в столбце связи одной таблицы со столбцом связи другой таблицы.  Наш оператор join установил критерии, согласно которым выходные данные должны включать только пары, чьи user_guid совпадают в двух связующих столбцах.  Однако, поскольку в таблице users было несколько строк с одинаковым user_guid, * каждая из этих строк была сопоставлена с каждой строкой в таблице dogs, которая имела одинаковый user_guid.*  Результатом было 442 строки, потому что 17 (экземпляры user_guid в таблице users) x 26 (экземпляры user_guid в таблице dogs) = 442.

Увидев это, возможно, теперь вы сможете понять, почему некоторые эксперты по базам данных уделяют особое внимание терминологии, которая проводит различие между теорией множеств и реальной реализацией базы данных.  Операции с базой данных, подобные тем, которые объединяют таблицы, основаны на теории множеств, которая предполагает, что в ваших таблицах нет повторяющихся строк.  В реальной жизни постоянно вводятся повторяющиеся строки, и это может привести к неправильной интерпретации ваших запросов, если вы не понимаете последствий.  Если вы достаточно часто сталкивались с различиями между реальными и теоретическими базами данных, имеет смысл, что для вас было бы важно использовать язык, который четко проводит различие между теорией и реальной жизнью.

Важные вещи, которые я хочу, чтобы вы запомнили из этого примера соединений с дубликатами, заключаются в том, что повторяющиеся строки и табличные отношения, которые имеют сопоставления таблица-таблица больше 1, оказывают мультипликативное влияние на результаты вашего запроса из-за способа объединения таблиц в реляционных базах данных.  Если вы пишете запросы, которые агрегируют данные по большому количеству объединенных таблиц, может быть очень трудно отследить проблемы, которые приводят к результатам, которые вы не предполагаете, потому что агрегированные результаты скроют от вас подсказки.   Чтобы этого не произошло, я рекомендую вам придерживаться следующих практик:

+ Избегайте делать предположения относительно ваших данных или результатов анализа.  Например, вместо того, чтобы предполагать, что все значения в столбце уникальны только потому, что в какой-то документации говорится, что так и должно быть, проверьте сами!
+ Всегда смотрите на примеры выходных данных ваших запросов, прежде чем строго интерпретировать агрегированные вычисления. Будьте особенно осторожны, когда ваши запросы требуют объединения.  
+ Если для ваших запросов требуется несколько уровней функций или объединений, сначала изучите выходные данные каждого уровня или объединения, прежде чем объединять их все вместе. 
+ Отнеситесь со здоровым скептицизмом ко всем вашим данным и результатам.  Если вы видите что-то, чего не ожидаете, обязательно изучите это, прежде чем давать строгую интерпретацию или включать в другие анализы.

Следует упомянуть еще один тип соединения - это полное внешнее соединение. Полные внешние соединения включают все строки в обеих таблицах в предложении ON, независимо от того, существует ли значение, которое связывает строку одной таблицы со строкой в другой таблице.  Как и в случае с левыми или правыми объединениями, всякий раз, когда значение в строке не имеет совпадающего значения в объединенной таблице, для всех значений в объединенной таблице будут введены нулевые значения.

Внешние соединения используются очень редко.  Наиболее практичное применение - это если вы хотите экспортировать все ваши необработанные данные в другую программу для визуализации или анализа. Синтаксис для внешних соединений такой же, как и для внутренних, но вы заменяете слово "внутренний" на "полный внешний":

```mysql
SELECT r.dog_guid AS rDogID, d.dog_guid AS dDogID, r.user_guid AS rUserID, d.user_guid AS dUserID, AVG(r.rating) AS AvgRating, COUNT(r.rating) AS NumRatings, d.breed, d.breed_group, d.breed_type
FROM reviews r FULL OUTER JOIN dogs d
  ON r.dog_guid=d.dog_guid AND r.user_guid=d.user_guid
WHERE r.dog_guid IS NOT NULL
GROUP BY r.dog_guid
ORDER BY AvgRating DESC;
```

**ОДНАКО!  MySQL не поддерживает полные внешние соединения.**


## Практика темы 7.

### Вопрос 7.

**Составте запрос, который использует LEFT JOIN для получения количества различных `user_guid`, которые были в таблице `users`, но не в таблице `dogs`**

### Вопрос 8.

**Составте запрос, который использует RIGHT JOIN для возврата количества различных `user_guid`, которые были в таблице `users`, но не в таблице `dogs`**

### Вопрос 9.

**Используйте LEFT JOIN, чтобы получить список уникальных идентификаторов `dog_guid`, которые содержатся в таблице `site_activities`, но не в таблице `dogs`, и сколько раз вводится каждый из них.  Обратите внимание, что в `dog_guid` таблицы `site_activities` много нулевых значений, поэтому вы захотите исключить их из своего списка.  (Подсказка: если вы исключите нулевые значения, в результатах, которые вы получите, будут две строки со словами в полях site_activities dog_guid вместо реальных guid из-за ошибочных записей)**